In [2]:
import pickle as pkl
import pandas as pd
import numpy as np
#import pytorch_nndct
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
#from pytorch_nndct import Inspector
batch_size = 1
# load data
data = pd.read_pickle("RML2016.10a_dict.pkl", compression='infer')
qpsk_2_data_all = data[('QPSK', 2)]
bpsk_2_data_all = data[('BPSK', 2)]

# labels
qpsk_labels = [0] * 1000  # QPSK = 0
bpsk_labels = [1] * 1000  # BPSK = 1

# combine the data lables
data_combined = np.concatenate((qpsk_2_data_all, bpsk_2_data_all), axis=0)
labels_combined = qpsk_labels + bpsk_labels

# convert labels to NumPy array and then to PyTorch tensor with Long data type
labels_combined = np.array(labels_combined, dtype=np.int64)
labels_combined = torch.from_numpy(labels_combined).long()

# convert 2 PyTorch tensor
data_combined = torch.from_numpy(data_combined).float()

# convert labels 2 NumPy array and then 2 PyTorch tensor
labels_combined = np.array(labels_combined)
labels_combined = torch.from_numpy(labels_combined)

# break into training + testing
test_size = 0.2  # Adjust the test size as needed
data_train, data_test, labels_train, labels_test = train_test_split(
    data_combined, labels_combined, test_size=test_size, random_state=42)

class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # access a single data sample and label
        sample = self.data[idx]
        label = self.labels[idx]
    
        # Convert sample, min_vals, and max_vals to PyTorch tensors
        sample = torch.tensor(sample, dtype=torch.float32)
        min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
        max_vals = torch.tensor(sample.max(axis=1).values, dtype=torch.float32)
        
        #normalize
        epsilon = 1e-10
        normalized_sample = 2 * (sample - min_vals.unsqueeze(1)) / (max_vals.unsqueeze(1) - min_vals.unsqueeze(1) + epsilon) - 1
    
        return normalized_sample, label

train_dataset = MyDataset(data_train, labels_train)
test_dataset = MyDataset(data_test, labels_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

/var/folders/j5/zgzptl2d205bwc_t75yd8fhh0000gn/T/ipykernel_18697/1501639799.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
"""class ModifiedToyModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.upsample = torch.nn.Upsample(scale_factor=2, mode='nearest')  # set mode = "nearest"
        self.conv = torch.nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.relu = torch.nn.ReLU()
        self.adaptive_avg_pool2d = torch.nn.AdaptiveAvgPool2d(output_size=1)
        
    def forward(self, x):
        x = self.upsample(x)
        x = self.conv(x)
        x = self.relu(x)
        x = self.adaptive_avg_pool2d(x)
        x = x.permute(0, 2, 3, 1) # insert a permute to cancel out the permute inserted by quantizer.
        x = x.reshape(x.size(0), -1)
        return x

# initialize model
num_classes = 2 #BPSK / QPSK
model = CNN1D(num_classes)
"""
import torch
import torch.nn as nn

class CNN2D(nn.Module):
    def __init__(self):
        super(CNN2D, self).__init__()
        
        in_channels = 2
        # For the 2D conversion, we're assuming height=1 for the input
        height = 1
        width = 128  # Original length dimension
        
        # Upsample (adjusted for 2D)
        self.upsample = nn.Upsample(scale_factor=(1, 2), mode='bilinear', align_corners=False)  # height stays the same, width is doubled
        
        # Define 2D convolutional layers
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=(1, 3), padding=(0, 1))
        self.relu1 = nn.ReLU()
        #self.maxpool1 = nn.MaxPool2d(kernel_size=(1, 2))
        self.adaptive_pool1 = nn.AdaptiveAvgPool2d((1,64))
        
        self.conv2 = nn.Conv2d(64, 128, kernel_size=(1, 3), padding=(0, 1))
        self.relu2 = nn.ReLU()
        #self.maxpool2 = nn.MaxPool2d(kernel_size=(1, 2))
        self.adaptive_pool2 = nn.AdaptiveAvgPool2d((1,32))
        
        # Adaptive pooling (2D)
        self.adaptive_avg_pool2d = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        
        # Define the fully connected layers (adjusted for adaptive pooling output)
        self.fc1 = nn.Linear(128, 256)  # The input features after pooling
        self.relu3 = nn.ReLU()
        #self.fc2 = nn.Linear(256, num_classes)
        self.fc2 = nn.Linear(256, 2) #binary classification (BPSK / QPSK) with 1 output neuron
        self.softmax = nn.Softmax(dim=1)
        
    
    def forward(self, x):
        # Adjust input shape for 2D processing
        x = x.unsqueeze(-2)  # Insert a height dimension of size 1
        
        # Upsample
        x = self.upsample(x)
        
        # 2D convolutional layers
        x = self.conv1(x)
        x = self.relu1(x)
        #x = self.maxpool1(x)
        x = self.adaptive_pool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        #x = self.maxpool2(x)
        x = self.adaptive_pool2(x)
        
        # Adaptive average pooling
        x = self.adaptive_avg_pool2d(x)
        
        # Flatten (before fully connected layers)
        x = x.reshape(x.size(0), -1)
        
        # Fully connected layers
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        x = self.softmax(x)
        
        return x


model = CNN2D()
print(model) #model info


CNN2D(
  (upsample): Upsample(scale_factor=(1.0, 2.0), mode='bilinear')
  (conv1): Conv2d(2, 64, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
  (relu1): ReLU()
  (adaptive_pool1): AdaptiveAvgPool2d(output_size=(1, 64))
  (conv2): Conv2d(64, 128, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
  (relu2): ReLU()
  (adaptive_pool2): AdaptiveAvgPool2d(output_size=(1, 32))
  (adaptive_avg_pool2d): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc1): Linear(in_features=128, out_features=256, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=256, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)


In [3]:
# Specify a target name or fingerprint you want to deploy on
#target = "DPUCAHX8L_ISA0_SP"
target = "DPUCVDX8G_ISA3_C32B3"
#DPUCVDX8G
# Initialize inspector with target
inspector = Inspector(target)
# Note: visualization of inspection results relies on the dot engine.If you don't install dot successfully, set 'image_format = None' when inspecting.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN2D(2)

#dummy_data, labels = train_loader
#data_dummy = []
for data, labels in train_loader:
    data_dummy = data
    break
print(type(data_dummy))
print(data_dummy.shape)
dummy_input = torch.randn(1, 2, 128)
print(dummy_input)

inspector.inspect(model, (data_dummy.clone().detach().requires_grad_(True)), device=device, output_dir="inspect", image_format=None) 


[VAIQ_NOTE]: Inspector is on.
<class 'torch.Tensor'>
torch.Size([1, 2, 128])
tensor([[[ 1.2419, -0.7442,  0.1608, -2.4847,  0.5102, -1.0885, -0.8666,
          -0.2171,  2.3965, -1.4778,  1.0408,  0.7034, -1.0147,  1.5584,
           0.2757,  0.0392, -0.4880,  1.2513,  1.0156,  0.1213,  0.3852,
           0.4313, -0.2750, -1.2605,  0.0075, -0.2101, -0.9361,  0.3707,
           1.6505, -0.2779, -0.6461,  0.6620, -2.1368, -0.4896, -0.7619,
          -1.1381, -0.4288,  0.5116,  0.5877,  0.8381,  0.2129, -0.5857,
          -0.6041,  0.1553, -0.2945,  1.7447,  0.8721, -1.6685,  0.3773,
           0.8339, -0.9704,  0.1246, -0.7313, -1.3037,  0.0848, -2.5284,
           1.5076, -0.0527,  0.4899,  0.2916, -1.2697, -0.2319, -0.3756,
          -1.5034,  1.0084, -1.2053,  0.0208, -0.0110,  0.5167, -0.3695,
           1.2093,  1.6096,  0.2915, -0.8964,  0.1353, -1.1268,  2.6418,
           0.3506, -0.3700, -1.0517,  0.3395,  0.6789,  1.7849,  1.0049,
           0.3596, -0.0507, -0.5034,  0.3627, 

/tmp/ipykernel_200599/822561179.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)
/tmp/ipykernel_200599/822561179.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
/tmp/ipykernel_200599/822561179.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_vals = torch.tensor(sample.max(axis=1).values, dtype=torch.float32)



[VAIQ_NOTE]: Start to trace and freeze model...

[VAIQ_NOTE]: The input model nndct_st_CNN2D_ed is torch.nn.Module.

[VAIQ_NOTE]: Finish tracing.

[VAIQ_NOTE]: Processing ops...


██████████████████████████████████████████████████| 16/16 [00:00<00:00, 1676.34it/s, OpInfo: name = return_0, type = Return]                                                    


[VAIQ_NOTE]: =>Doing weights equalization...

[VAIQ_NOTE]: =>Quantizable module is generated.(inspect/CNN2D.py)

[VAIQ_WARN]: CNN2D::507 is not tensor.

[VAIQ_NOTE]: Find subgraph for convlike_fix_18:
node name:CNN2D::CNN2D/Linear[fc1]/ret.19, op type:nndct_dense, output shape: [1, 256]
node name:CNN2D::CNN2D/ReLU[relu3]/ret.21, op type:nndct_relu, output shape: [1, 256]



[VAIQ_NOTE]: Find subgraph for convlike_fix_18:
node name:CNN2D::CNN2D/Conv2d[conv2]/ret.11, op type:nndct_conv2d, output shape: [1, 1, 64, 128]
node name:CNN2D::CNN2D/ReLU[relu2]/ret.13, op type:nndct_relu, output shape: [1, 1, 64, 128]



[VAIQ_NOTE]: Find subgraph for convlike_fix_18:
node name:CNN2D::CNN2D/Conv2d[conv1]/ret.7, op type:nndct_conv2d, output shape: [1, 1, 256, 64]
node name:CNN2D::CNN2D/ReLU[relu1]/ret.9, op type:nndct_relu, output shape: [1, 1, 256, 64]



[VAIQ_NOTE]: Find subgraph for reshape_fix_1:
node name:CNN2D::CNN2D/ret.17, op type:nndct_reshape, output shape: [1, 128]



[VAIQ_NOTE]: Fin


I20240229 09:17:56.093032 200599 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20240229 09:17:56.093055 200599 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20240229 09:17:56.093060 200599 compile_pass_manager.cpp:357] [UNILOG][INFO] Target architecture: DPUCVDX8G_ISA3_C32B3
I20240229 09:17:56.093122 200599 compile_pass_manager.cpp:465] [UNILOG][INFO] Graph name: nndct_dense_nndct_relu_Zq6mEnaGWUs0rhFH, with op num: 9
I20240229 09:17:56.093125 200599 compile_pass_manager.cpp:478] [UNILOG][INFO] Begin to compile...
I20240229 09:17:56.097330 200599 compile_pass_manager.cpp:489] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20240229 09:17:56.097349 200599 compile_pass_manager.cpp:504] [UNILOG][INFO] Compile done.
I20240229 09:17:56.099707 200599 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20240229 09:17:56.099715 200599 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20240229 09:17:56.099718 200

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


learning_rate = 0.001
num_epochs = 3

# loss function and optimizer definition
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# training
for epoch in range(num_epochs):
    model.train()  # Set the model in training mode
    
    for data, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(data)  # Forward pass
        loss = criterion(outputs, labels)  # Calculate the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
    
    # print loss
    print(f'Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item():.4f}')

def evaluate (model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, labels in test_loader:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # predicted and actual labels
            for i in range(len(labels)):
                print(f'Predicted: {predicted[i]}, Actual: {labels[i]}')
            accuracy = 100 * correct / total
        print(f'Test Accuracy: {accuracy:.2f}%')
    return accuracy
accuracy_orig_model = evaluate(model)


/var/folders/j5/zgzptl2d205bwc_t75yd8fhh0000gn/T/ipykernel_18697/1501639799.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)
/var/folders/j5/zgzptl2d205bwc_t75yd8fhh0000gn/T/ipykernel_18697/1501639799.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
/var/folders/j5/zgzptl2d205bwc_t75yd8fhh0000gn/T/ipykernel_18697/1501639799.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_vals = torch.tensor(sample.max

Epoch [1/3] Loss: 0.3153
Epoch [2/3] Loss: 0.3139
Epoch [3/3] Loss: 0.3133
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 0, Actual: 1
Predicted: 0, Actual: 0
Predicted: 0, Actual: 1
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 0,

In [7]:
print(total)

NameError: name 'total' is not defined

In [7]:
torch.save(model, "trainedModel")

In [9]:
from pytorch_nndct.apis import torch_quantizer, dump_xmodel



quantizer = torch_quantizer("calib", model, (dummy_input))
quant_model = quantizer.quant_model # is quantize aware

#accl_gen, acc4_gen, loss_get = evaluate(quant_model, val_loader, criterion)
# evaluate the model on test data
accuarcy_quant_model = evaluate(quant_model)


[VAIQ_WARN][QUANTIZER_TORCH_CUDA_UNAVAILABLE]: CUDA (HIP) is not available, change device to CPU

[VAIQ_NOTE]: OS and CPU information:
               system --- Linux
                 node --- seakn-ThinkPad-P16s-Gen-1
              release --- 6.5.0-17-generic
              version --- #17~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Jan 16 14:32:32 UTC 2
              machine --- x86_64
            processor --- x86_64

[VAIQ_NOTE]: Tools version information:
                  GCC --- GCC 7.5.0
               python --- 3.8.6
              pytorch --- 1.13.1
        vai_q_pytorch --- 3.5.0+60df3f1+torch1.13.1

[VAIQ_WARN][QUANTIZER_TORCH_CUDA_UNAVAILABLE]: CUDA (HIP) is not available, change device to CPU.

[VAIQ_NOTE]: Quant config file is empty, use default quant configuration

[VAIQ_NOTE]: Quantization calibration process start up...

[VAIQ_NOTE]: =>Quant Module is in 'cpu'.

[VAIQ_NOTE]: =>Parsing CNN2D...

[VAIQ_NOTE]: Start to trace and freeze model...

[VAIQ_NOTE]: The input model 

██████████████████████████████████████████████████| 16/16 [00:00<00:00, 2723.35it/s, OpInfo: name = return_0, type = Return]                                                    


[VAIQ_NOTE]: =>Doing weights equalization...

[VAIQ_NOTE]: =>Quantizable module is generated.(quantize_result/CNN2D.py)

[VAIQ_NOTE]: =>Get module with quantization.
Predicted: 1, Actual: 1
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 1, Actual: 1
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 1, Actual: 1
Test Accuracy: 100.00%
Predicted: 1, Actual: 1
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 1, Actual: 1
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 1, Actual: 1
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 1, Actual: 1
Test Accuracy: 100.00%
Predicted: 1, Actual: 1
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accur


/tmp/ipykernel_200599/822561179.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)
/tmp/ipykernel_200599/822561179.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
/tmp/ipykernel_200599/822561179.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_vals = torch.tensor(sample.max(axis=1).values, dtype=torch.float32)


Predicted: 1, Actual: 1
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 1, Actual: 1
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 1, Actual: 1
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 1, Actual: 1
Test Accuracy: 100.00%
Predicted: 1, Actual: 1
Test Accuracy: 100.00%
Predicted: 1, Actual: 1
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 1, Actual: 1
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 0, Actual: 0
Test Accuracy: 100.00%
Predicted: 1, Actual: 1
Test Accuracy: 100.00%
Predicted: 0,

Predicted: 0, Actual: 0
Test Accuracy: 94.61%
Predicted: 1, Actual: 0
Test Accuracy: 94.15%
Predicted: 0, Actual: 0
Test Accuracy: 94.17%
Predicted: 1, Actual: 1
Test Accuracy: 94.20%
Predicted: 1, Actual: 1
Test Accuracy: 94.23%
Predicted: 1, Actual: 1
Test Accuracy: 94.26%
Predicted: 0, Actual: 0
Test Accuracy: 94.29%
Predicted: 1, Actual: 1
Test Accuracy: 94.31%
Predicted: 0, Actual: 0
Test Accuracy: 94.34%
Predicted: 0, Actual: 0
Test Accuracy: 94.37%
Predicted: 0, Actual: 0
Test Accuracy: 94.39%
Predicted: 1, Actual: 0
Test Accuracy: 93.95%
Predicted: 1, Actual: 1
Test Accuracy: 93.98%
Predicted: 1, Actual: 1
Test Accuracy: 94.01%
Predicted: 0, Actual: 0
Test Accuracy: 94.04%
Predicted: 0, Actual: 0
Test Accuracy: 94.06%
Predicted: 1, Actual: 1
Test Accuracy: 94.09%
Predicted: 0, Actual: 0
Test Accuracy: 94.12%
Predicted: 0, Actual: 0
Test Accuracy: 94.14%
Predicted: 0, Actual: 0
Test Accuracy: 94.17%
Predicted: 1, Actual: 1
Test Accuracy: 94.20%
Predicted: 1, Actual: 1
Test Accur

Predicted: 0, Actual: 0
Test Accuracy: 93.73%
Predicted: 1, Actual: 1
Test Accuracy: 93.75%
Predicted: 1, Actual: 1
Test Accuracy: 93.77%
Predicted: 0, Actual: 0
Test Accuracy: 93.78%
Predicted: 0, Actual: 0
Test Accuracy: 93.80%
Predicted: 1, Actual: 1
Test Accuracy: 93.81%
Predicted: 0, Actual: 0
Test Accuracy: 93.83%
Predicted: 1, Actual: 1
Test Accuracy: 93.85%
Predicted: 1, Actual: 1
Test Accuracy: 93.86%
Predicted: 1, Actual: 1
Test Accuracy: 93.88%
Predicted: 0, Actual: 0
Test Accuracy: 93.89%
Predicted: 0, Actual: 0
Test Accuracy: 93.91%
Predicted: 0, Actual: 0
Test Accuracy: 93.92%
Predicted: 0, Actual: 0
Test Accuracy: 93.94%
Predicted: 1, Actual: 1
Test Accuracy: 93.95%
Predicted: 1, Actual: 1
Test Accuracy: 93.97%
Predicted: 0, Actual: 0
Test Accuracy: 93.98%
Predicted: 0, Actual: 0
Test Accuracy: 94.00%


In [14]:
quantizer.export_quant_config() # creates quantizer config.json

quantizer.export_torch_script #exports [model_name]_int.pt
quantizer.export_onnx_model() #exports [model_name]_int.onnx

quantizer.export_xmodel(deploy_check=True) # in therory exports xmodel, but exports [model_name].py



[VAIQ_NOTE]: =>Exporting quant config.(quantize_result/quant_info.json)

[VAIQ_NOTE]: CNN2D_int.pt is generated.(quantize_result/CNN2D_int.pt)

[VAIQ_NOTE]: CNN2D_int.onnx is generated.(quantize_result/CNN2D_int.onnx)
